## CatBoost

In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report, roc_auc_score,f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve

In [ ]:
# load the dataset
Train = pd.read_csv("MergedData.csv")
Train # show some sample data

In [ ]:
Train['subcategory'].value_counts(dropna=False)

In [ ]:
# convert attack types from labels to integers
Train['subcategory'].replace({
    'Normal': 0,
    'DDH': 1,
    'DDP': 2,
    'DDT': 3,
    'DH': 4, 
    'DP': 5, 
    'DT': 6, 
    'Data_Exfiltration': 7, 
    'Keylogging': 8, 
    'OS_Fingerprint': 9, 
    'Service_Scan': 10
}, inplace=True)

In [ ]:
# split the dataset for training and testing
from sklearn.model_selection import train_test_split
X = Train.drop('subcategory', axis = 1)
Y = Train.subcategory
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [ ]:
# apply MinMax Scaler
from sklearn.preprocessing import MinMaxScaler
ma = MinMaxScaler()
ma.fit(xtrain)
xtrain = ma.transform(xtrain)
xtest = ma.transform(xtest)

In [ ]:
# import CatBoost
import catboost as ctb
dt = ctb.CatBoostClassifier(iterations=20)

In [ ]:
# training
start_time = datetime.now()
dt.fit(xtrain, ytrain)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
# testing
start_time = datetime.now()
dtpred = dt.predict(xtest)
dtpred_proba = dt.predict_proba(xtest)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
# evaluation
print(confusion_matrix(ytest, dtpred))
sk_report = classification_report(digits=4,y_true=ytest,y_pred=dt.predict(xtest))
print(sk_report)
print("Accuracy: %.7f%%" % (dt.score(xtest, ytest)*100.0))